In [11]:
import optuna

In [12]:
best_trials = {
    "lightgbm_with_resampling": 248,
    "lightgbm_without_resampling": 220,
    "svc_without_resampling": 203,
    "svc_with_resampling": 161,
    "catboost_with_resampling": 234,
    "catboost_without_resampling": 223,
    "fastai_with_resampling": 123,
    "fastai_without_resampling": 101,
    "tabpfn_without_resampling": 30,
    "tabpfn_with_resampling": 65,
    "xgboost_without_resampling": 210,
    "xgboost_with_resampling": 179,
}

In [13]:
from pathlib import Path
path = Path('./data')
output = Path('./output')
output.mkdir(exist_ok=True)

In [14]:
optuna_storage = "sqlite:///icr-ensemble-experiments.db"

In [17]:
estimator_params = {}
for study_name, trial_id in best_trials.items():
    study = optuna.load_study(study_name=study_name, storage=optuna_storage)
    best_trial = best_trials[study_name]
    estimator_params[study_name] = study.trials[trial_id].params

{'colsample_bynode': 1.0,
 'colsample_bytree': 0.9,
 'learning_rate': 0.05483336534261373,
 'max_bin': 348,
 'max_depth': 4,
 'n_estimators': 450,
 'num_leaves': 20146,
 'reg_alpha': 2.5,
 'reg_lambda': 1.5,
 'subsample': 0.4}